In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
PATH = '/kaggle/input/tabular-playground-series-oct-2021/'
train = pd.read_csv(f'{PATH}train.csv')
test = pd.read_csv(f'{PATH}test.csv')
sub = pd.read_csv(f'{PATH}sample_submission.csv')

In [ ]:
print(f'Number of missing values in training data: {train.isna().sum().sum()}')
print(f'Number of missing values in testing data: {test.isna().sum().sum()}')

In [ ]:
train.info(memory_usage="deep")

In [ ]:
#Garbage Collector
import gc
#This is a function that downcast the integer columns
def downcast_df_int_columns(df):
    list_of_columns = list(df.select_dtypes(include=["int32", "int64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting integers for:", list_of_columns, "\n")
        
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="integer")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
    
    gc.collect()
    
    print("done")

In [ ]:
#This is a function that downcast the float columns, if you have too many columns to adjust and do not want to see to many messages proceesing, you could comment our the print() columns
def downcast_df_float_columns(df):
    list_of_columns = list(df.select_dtypes(include=["float64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        print("downcasting float for:", list_of_columns, "\n")
        
        for col in list_of_columns:
            print("reduced memory usage for:  ", col.ljust(max_string_length+2)[:max_string_length+2],
                  "from", str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8), "to", end=" ")
            df[col] = pd.to_numeric(df[col], downcast="float")
            print(str(round(df[col].memory_usage(deep=True)*1e-6,2)).rjust(8))
    else:
        print("no columns to downcast")
    
    gc.collect()
    
    print("done")

In [ ]:
downcast_df_int_columns(train)
downcast_df_float_columns(train)

In [ ]:
train.info(memory_usage="deep")

In [ ]:
test.info(memory_usage="deep")

In [ ]:
downcast_df_int_columns(train)
downcast_df_float_columns(train)

In [ ]:
test.info(memory_usage="deep")

In [ ]:
features = [col for col in train.columns if 'f' in col]
features.append('id')


In [ ]:
len(features)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_frac = train.sample(frac = 1).reset_index(drop = True)

In [ ]:
test_frac = test.sample(frac = 1).reset_index(drop = True)

In [ ]:
# plot the first 5 features 
# i = 1
# plt.figure()
# fig, ax = plt.subplots(285, 5,figsize=(30, 285))
# for feature in features:
#     plt.subplot(285, 5,i)
#     sns.histplot(train_frac[feature],color="blue", kde=True,bins=100, label='train_'+feature)
#     sns.histplot(test_frac[feature],color="olive", kde=True,bins=100, label='test_'+feature)
#     plt.xlabel(feature, fontsize=9); plt.legend()
#     i += 1
# plt.show()

In [ ]:
# import pandas_profiling as pp
# pp.ProfileReport(train_frac)

In [ ]:
# train['std'] = train[features].std(axis=1)

In [ ]:
#Check scale
train_frac.describe().T.style.bar(subset=['mean'], color='#FF595E')\
                           .background_gradient(subset=['50%'], cmap='PiYG')

In [ ]:
from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()
# X_train[features] = scaler.fit_transform(X_train[features])


In [ ]:
# Library
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

# train_X, val_X, train_y, val_y = train_test_split(X_train, y, train_size=0.8, test_size=0.2,
#                                                                 random_state=0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score

In [ ]:
# model = lgbm.LGBMClassifier()
# model.fit(
#         train_X, 
#         train_y,
#         eval_set=[(val_X, val_y)],
#         eval_metric='auc',
#         early_stopping_rounds=200,
#         verbose=1000,
#     )

# lgb_oof = model.predict_proba(val_X)[:,-1]

# print(roc_auc_score(val_y, lgb_oof))

# tmp = pd.DataFrame()
# tmp['imp'] = model.feature_importances_
# tmp['feature'] = model.feature_name_
# tmp_or = tmp
# tmp = tmp[tmp.imp > 0]
# order = list(tmp.groupby('feature').mean().sort_values('imp', ascending=False).index)
# fig = plt.figure(figsize=(16, 16), tight_layout=True)
# sns.barplot(x="imp", y="feature", data=tmp.groupby('feature').mean().reset_index(), order=order)
# plt.title("LightGBM feature importances")


In [ ]:
train_or = train
test_or = test
train = train_frac
test = test_frac
target = train.target

In [ ]:
train = train[features].set_index('id')
test = test[features].set_index('id')

features.remove('id')

train['std'] = train[features].std(axis=1)
train['min'] = train[features].min(axis=1)
train['median'] = train[features].median(axis=1)
train['max'] = train[features].max(axis=1)
train['mean'] = train[features].mean(axis=1)

test['std'] = test[features].std(axis=1)
test['min'] = test[features].min(axis=1)
test['median'] = test[features].median(axis=1)
test['max'] = test[features].max(axis=1)
test['mean'] = test[features].mean(axis=1)

features.extend(['std','min','median','max','mean'])

In [ ]:
scaler = RobustScaler()
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

# Optuna Training

In [ ]:
N_SPLITS = 5
N_ESTIMATORS = 20000
EARLY_STOPPING_ROUNDS = 200
VERBOSE = 1000
SEED = 2021

import pandas as pd
import numpy as np
import random
import time
import os
import gc

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

In [ ]:

from optuna.integration import LightGBMPruningCallback
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
def objective(trial, train, target):
    param_grid = {
        "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 5e-4, 0.5),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
        'subsample': trial.suggest_float(
            "subsample", 0.1, 1, step=0.1
        ),
        'subsample_freq': trial.suggest_int('subsample_freq',0,1),
        'colsample_bytree':trial.suggest_float(
            "colsample_bytree", 0, 10),
        'reg_alpha': trial.suggest_float(
            "reg_alpha", 1.0, 20),
        'reg_lambda': trial.suggest_float(
            "reg_lambda", 1e-2, 5),
        'min_child_weight': trial.suggest_int("min_child_weight", 0, 500),
        'min_child_samples': trial.suggest_int("min_child_samples", 20, 1000)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(train, target)):
        X_train, X_test = train.iloc[train_idx], train.iloc[test_idx]
        y_train, y_test = target[train_idx], target[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="auc",
            early_stopping_rounds=100
        )
        preds = model.predict_proba(X_test)[:,1]
        cv_scores[idx] = roc_auc_score(y_test, preds)

    return np.mean(cv_scores)
    
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, train, target)
study.optimize(func, n_trials=20)

In [ ]:
print(f"\tBest value (auc): {study.best_value:.5f}") 


print(f"\tBest params:")

best_params = study.best_params


In [ ]:
best_params

# Training Models

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)

In [ ]:
# best_params = {'device_type': 'gpu',
#  'n_estimators': 10000,
#  'learning_rate': 0.005561317553182468,
#  'num_leaves': 20,
#  'max_depth': 5,
#  'min_data_in_leaf': 9400,
#  'lambda_l1': 0,
#  'lambda_l2': 95,
#  'min_gain_to_split': 4.175935643818434,
#  'bagging_fraction': 0.9,
#  'bagging_freq': 1,
#  'feature_fraction': 0.30000000000000004,
#  'subsample': 0.6,
#  'subsample_freq': 1,
#  'colsample_bytree': 9.959598465442731,
#  'reg_alpha': 11.705140163646389,
#  'reg_lambda': 3.897135186715671,
#  'min_child_weight': 266,
#  'min_child_samples': 985}

In [ ]:
lgb_oof = np.zeros(train.shape[0])
lgb_pred = np.zeros(test.shape[0])
lgb_importances = pd.DataFrame()

for fold, (trn_idx, val_idx) in enumerate(skf.split(X=train, y=target)):
    print(f"===== fold {fold} =====")
    X_train = train[features].iloc[trn_idx]
    y_train = target.iloc[trn_idx]
    X_valid = train[features].iloc[val_idx]
    y_valid = target.iloc[val_idx]
    X_test = test[features]
    
    start = time.time()
    model = lgbm.LGBMClassifier(**best_params)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='auc',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
    )
    
    fi_tmp = pd.DataFrame()
    fi_tmp['feature'] = model.feature_name_
    fi_tmp['importance'] = model.feature_importances_
    fi_tmp['fold'] = fold
    fi_tmp['seed'] = SEED
    lgb_importances = lgb_importances.append(fi_tmp)

    lgb_oof[val_idx] = model.predict_proba(X_valid)[:, -1]
    lgb_pred += model.predict_proba(X_test)[:, -1] / N_SPLITS

    elapsed = time.time() - start
    auc = roc_auc_score(y_valid, lgb_oof[val_idx])
    print(f"fold {fold} - lgb auc: {auc:.6f}, elapsed time: {elapsed:.2f}sec\n")

print(f"oof lgb roc = {roc_auc_score(target, lgb_oof)}")

np.save("lgb_oof.npy", lgb_oof)
np.save("lgb_pred.npy", lgb_pred)

In [ ]:
order = list(lgb_importances.groupby('feature').mean().sort_values('importance', ascending=False).index)

fig = plt.figure(figsize=(16, 120), tight_layout=True)
sns.barplot(x="importance", y="feature", data=lgb_importances.groupby('feature').mean().reset_index(), order=order)
plt.title("LightGBM feature importances")

In [ ]:
# best_params['learning_rate'] = 5e-3
# Didn't do any good

In [ ]:
# lgb_oof = np.zeros(train.shape[0])
# lgb_pred = np.zeros(test.shape[0])
# lgb_importances = pd.DataFrame()

# for fold, (trn_idx, val_idx) in enumerate(skf.split(X=train, y=target)):
#     print(f"===== fold {fold} =====")
#     X_train = train[features].iloc[trn_idx]
#     y_train = target.iloc[trn_idx]
#     X_valid = train[features].iloc[val_idx]
#     y_valid = target.iloc[val_idx]
#     X_test = test[features]
    
#     start = time.time()
#     model = lgbm.LGBMClassifier(**best_params)
#     model.fit(
#         X_train, 
#         y_train,
#         eval_set=[(X_valid, y_valid)],
#         eval_metric='auc',
#         early_stopping_rounds=EARLY_STOPPING_ROUNDS,
#         verbose=VERBOSE,
#     )
    
#     fi_tmp = pd.DataFrame()
#     fi_tmp['feature'] = model.feature_name_
#     fi_tmp['importance'] = model.feature_importances_
#     fi_tmp['fold'] = fold
#     fi_tmp['seed'] = SEED
#     lgb_importances = lgb_importances.append(fi_tmp)

#     lgb_oof[val_idx] = model.predict_proba(X_valid)[:, -1]
#     lgb_pred += model.predict_proba(X_test)[:, -1] / N_SPLITS

#     elapsed = time.time() - start
#     auc = roc_auc_score(y_valid, lgb_oof[val_idx])
#     print(f"fold {fold} - lgb auc: {auc:.6f}, elapsed time: {elapsed:.2f}sec\n")

# print(f"oof lgb roc = {roc_auc_score(target, lgb_oof)}")

# np.save("lgb_oof.npy", lgb_oof)
# np.save("lgb_pred.npy", lgb_pred)

In [ ]:
# lgb_pred = model.predict_proba(test)[:,-1]

In [ ]:
# sub['target'] = lgb_pred
# sub.to_csv("submission.csv", index=False)
# sub